In [ ]:
!pip install image-classifiers==1.0.0b1
!pip install keras_applications --no-deps
!pip install tensorflow==2.10

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.regularizers import l2
print(tf.__version__)

2.10.0


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Load Adience dataset (with newly encoded age)

In [4]:
fold0_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold0_new')
fold1_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold1_new')
fold2_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold2_new')
fold3_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold3_new')
fold4_new = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold4_new')
train_new = fold0_new.concatenate(fold1_new)
train_new = train_new.concatenate(fold2_new)
val_new = fold3_new
val_new_batch = val_new.batch(1)
test_new = fold4_new
test_new_batch = test_new.batch(1)

### Load base model pretrained on CELEBA

In [ ]:
base_model = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')

- We want to add layers to CELEBA model, so we do not use the model output, we use the layer before all output layers (named 'relu1')
- We create a `new_base_model` with same input but use 'relu1' as output, then add layers we want to this `new_base_model` below

In [ ]:
base_model.layers[-42] # this is where 'relu1' is since last 40 are output layers and last 41 is pooling

In [ ]:
# new_base_model with 'relu1' as output
new_base_model = tf.keras.Model(inputs = base_model.input,
                                outputs = [base_model.get_layer('relu1').output])

# iterate over all the layers in orignal base_model to load weights for new_base_model
weights = [layer.get_weights() for layer in base_model.layers[:-42]]
for layer, weight in zip(new_base_model.layers, weights):
  layer.set_weights(weight)

- We want to freeze all layers except the last 10 layers. <br> Hence, we make all layers except the last 10 layers non-trainable

In [ ]:
for layer in new_base_model.layers[:-10]:
  layer.trainable = False

### Create model for Adience based on pretrained model

In [ ]:
def create_model(base_model):
  ''' 
  Add squeeze-excite block on top of last layer before classificatiion layer

  Args:
        base_model: ResNet34 model pretrained on CELEBA
        
  Returns:
        new model used to train age_gender
  '''


  img = tf.keras.Input(shape=(224,224,3))

  # add data augmentation
  img = tf.keras.layers.RandomFlip(mode='horizontal_and_vertical')(img)
  img = tf.keras.layers.RandomRotation(0.1)(img)
  img = tf.keras.layers.RandomContrast(0.1)(img)

  # pass the image through base model trained on CELEBA
  pretrained_output = base_model(img)

  # squeeze
  main_branch = tf.keras.layers.GlobalAveragePooling2D()(pretrained_output)
  # excite
  main_branch = tf.keras.layers.Dense(256, activation = 'relu')(main_branch)
  main_branch = tf.keras.layers.Dense(512, activation='sigmoid')(main_branch)
  # scale back to same dim
  main_branch = tf.keras.layers.Multiply()([pretrained_output, main_branch])
  main_branch = tf.keras.layers.GlobalAveragePooling2D()(main_branch)
  # Dropout layer to prevent overfitting
  main_branch = tf.keras.layers.Dropout(0.6)(main_branch)

  # add Dense before age and gender output
  gender_branch = tf.keras.layers.Dense(128, activation='relu')(main_branch)
  gender_branch = tf.keras.layers.Dropout(0.8)(gender_branch)
  gender_branch = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='gender_output')(gender_branch)
  
  age_branch = tf.keras.layers.Dense(128, activation='relu')(main_branch)
  age_branch = tf.keras.layers.Dropout(0.8)(age_branch)
  age1 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group1')(age_branch)
  age2 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group2')(age_branch)
  age3 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group3')(age_branch)
  age4 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group4')(age_branch)
  age5 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group5')(age_branch)
  age6 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group6')(age_branch)
  age7 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group7')(age_branch)
  age8 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='age_group8')(age_branch)
  model = tf.keras.Model(inputs = img,
                         outputs = [gender_branch, age1, age2, age3, age4, age5, age6, age7, age8])

  return model

In [ ]:
model = create_model(new_base_model)

### Evaluate age accuracy:
- The model has 8 outputs for age, each being whether 'the age of this image is older than {age_class}'. We combine label/prediction to give each image an age vector eg. `[1, 1, 0, 0, 0, 0, 0, 0]` if it belongs to the second age_group
- To do so, for predictions
```python
pred = [[[0.8], [0.3], ..., [last_in_batch]], # all outputs for gender in batch
          [[0.3], [0.2], ..., [last_in_batch]], # all outputs for age_group1 in batch
          [[0.6], [0.8], ..., [last_in_batch]], # all outputs for age_group2 in batch
          ...]
# We need all predictions from index 1 onwards (first one is prediction for gender), 
# then reshape each age element in predictions to have shape (batch_size,)
pred become: [[0.3, 0.2, ..., last_in_batch], # all outputs for age_group1
                [0.6, 0.8, ..., last_in_batch], # all outputs for age_group2
                ...]
# None each column is age feature for one image, we transpose it to make it take a row each
pred become: [[0.3, 0.2, ..., age_group8_output], 
                [0.6, 0.8, ..., age_group8_output], 
                ...
                [last_in_batch...................]]
# We then turn it into 0/1 binary values with threshold 0.5
pred become: [[0, 0, ..., age_group8_output], 
                [1, 1, ..., age_group8_output], 
                ...
                [last_in_batch...................]]
=> age_vec == [0, 0, ...] # length = 8
```
- for labels
```python
# We iterative over the whole dataset one example at a time. each example’s label_dict:
label_dict = {'age_group1':[[1]], 'age_group2': [[0]], ...}
# Hence, we take each value of 'age_groupi' and turn it to an age vector
=> age_vec == [1, 0, ...]
```
- accuracy calculation <br>
We only treat exact same age vector as a correct prediction, and we use sklearn accuracy score to compute this


In [5]:
from sklearn.metrics import accuracy_score

def decode_age_pred(pred):
  # tranpose to a 2D array with each row being an age_vector of an image
  # procedures described above
  age_logits = np.array([i.ravel() for i in pred[1:]]).transpose() 

  # encode sigmoid output to 0/1
  age_pred = np.where(age_logits > 0.5, 1, 0) 
  return age_pred

def decode_age_truth(ds):
  age_true = []
  for i in ds.as_numpy_iterator():
    # take label dict
    label_dict = i[1] 
    # append all values with key 'age_group_i' to a list, which is the age_vector
    age = [label_dict['age_group'+str(a)][0][0] for a in range(1,9)] 
    # append all age_vector to a list
    age_true.append(age) 
  return np.array(age_true)

def age_acc(model, val_ds):
  truths = decode_age_truth(val_ds)
  preds = model.predict(val_ds, verbose = 0)
  preds = decode_age_pred(preds)
  return accuracy_score(truths, preds)

# add above age accuracy score to log via callback
class AgeAccCallback(Callback):
  def __init__(self, val_ds, logs = {}):
    self.val_ds = val_ds
  def on_epoch_end(self, epoch, logs={}):
    logs['overall_age_accuracy'] = age_acc(self.model, self.val_ds)

### Train model
- We noticed that since age has multiple outputs, if 'gender_loss_weight' is set to 1, the model cannot learn to predict gender (accuracy around 0.5) since too much loss are generated by age and the model tend to focus on that
- Hence, we decided to set 'gender_loss_weight' higher at the first few epochs, 'gender_accuracy' will thus reach to an optimal level very soon. After that, we set 'gender_loss_weight' to a small value to let the model focus more on optimizing age classification
- Since tensorflow does not provide functions to change `loss_weights` while training, we try to mimic this process using a for loop:
  - fit the model with only 1 epoch
  - compile again at the beginning of each iteration to reset `loss_weights`
- Model checkpoint cannot be used since we only fit 1 epoch each time. Hence, we manully monitor 'mean_val_age_gender_accuracy' and save the model when it increases

In [ ]:
old_mean_acc = -1
gen_acc = -1
file_path = "/content/drive/MyDrive/data/saved_model/celeba_modified_age_gen_plus.h5"

for epoch_idx in range(1, 51):
  # for each iteration, set new config and compile

  # train epoch 11~50 or after gen_acc > 0.9, with this config
  if epoch_idx > 10 or gen_acc > 0.9:
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.0003),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 0.5, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )

  # train epoch 1~10 with this config
  else:
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.0007),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 2, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )

  # train newly compiled model 1 epoch
  print('Epoch', str(epoch_idx)+'/50')
  hist = model.fit(train_new.shuffle(1024).batch(64),
                   epochs=1,
                   validation_data = val_new.shuffle(1024).batch(64),
                   callbacks = [EarlyStopping(monitor = 'val_loss', patience=5),
                                AgeAccCallback(val_new_batch),])
  
  # check whether metrics improved and save model
  new_mean_acc = (hist.history['val_gender_output_accuracy'][0] + hist.history['overall_age_accuracy'][0])/2
  gen_acc = hist.history['gender_output_accuracy'][0]
  if new_mean_acc > old_mean_acc:
    print('Validation mean accuracy for gender and age improved to ' + str(new_mean_acc) + ', saving model...')
    model.save(file_path)
    old_mean_acc = new_mean_acc

Epoch 1/50
118/118 [==============================] - 110s 877ms/step - loss: 8.4886 - gender_output_loss: 0.7205 - age_group1_loss: 0.2261 - age_group2_loss: 0.4460 - age_group3_loss: 0.6253 - age_group4_loss: 0.7180 - age_group5_loss: 0.7172 - age_group6_loss: 0.6357 - age_group7_loss: 0.4468 - age_group8_loss: 0.3623 - gender_output_accuracy: 0.5430 - age_group1_accuracy: 0.9481 - age_group2_accuracy: 0.8404 - age_group3_accuracy: 0.6837 - age_group4_accuracy: 0.5855 - age_group5_accuracy: 0.5656 - age_group6_accuracy: 0.6904 - age_group7_accuracy: 0.8384 - age_group8_accuracy: 0.8822 - val_loss: 6.6620 - val_gender_output_loss: 0.6802 - val_age_group1_loss: 0.0753 - val_age_group2_loss: 0.2349 - val_age_group3_loss: 0.4254 - val_age_group4_loss: 0.5914 - val_age_group5_loss: 0.6388 - val_age_group6_loss: 0.5531 - val_age_group7_loss: 0.2723 - val_age_group8_loss: 0.2172 - val_gender_output_accuracy: 0.5447 - val_age_group1_accuracy: 1.0000 - val_age_group2_accuracy: 0.9771 - val_ag

#### We also tried not freezing any layers in CELEBA model (make all layers trainable) and this is the result:

In [ ]:
base_model2 = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')
new_base_model2 = tf.keras.Model(inputs = base_model2.input,
                                 outputs = [base_model2.get_layer('relu1').output])

# iterate over all the layers in orignal base_model to load weights for new_base_model
weights = [layer.get_weights() for layer in base_model2.layers[:-42]]
for layer, weight in zip(new_base_model2.layers, weights):
  layer.set_weights(weight)

In [ ]:
model2 = create_model(new_base_model2)

In [ ]:
old_mean_acc = -1
gen_acc = -1
file_path = "/content/drive/MyDrive/save_model/full_celeba_modified_age_gen_plus.h5"

for epoch_idx in range(1, 51):
  # for each iteration, set new config and compile

  # train epoch 11~50 or after gen_acc > 0.9, with this config
  if epoch_idx > 10 or gen_acc > 0.9:
    model2.compile(
        optimizer=tf.keras.optimizers.Adam(0.0003),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 0.5, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )

  # train epoch 1~10 with this config
  else:
    model2.compile(
        optimizer=tf.keras.optimizers.Adam(0.0007),
        loss={'gender_output': 'binary_crossentropy', 
              'age_group1': 'binary_crossentropy', 'age_group2': 'binary_crossentropy', 'age_group3': 'binary_crossentropy', 
              'age_group4': 'binary_crossentropy', 'age_group5': 'binary_crossentropy', 'age_group6': 'binary_crossentropy',
              'age_group7': 'binary_crossentropy', 'age_group8': 'binary_crossentropy'},
        loss_weights = {'gender_output': 2, 
                        'age_group1': 1, 'age_group2': 1, 'age_group3': 1, 
                        'age_group4': 1.3, 'age_group5': 1.8, 'age_group6': 1.8,
                        'age_group7': 1.3, 'age_group8': 1},  
        metrics=['accuracy'],
    )

  # train newly compiled model 1 epoch
  print('Epoch', str(epoch_idx)+'/50')
  hist = model2.fit(train_new.shuffle(1024).batch(64),
                    epochs=1,
                    validation_data = val_new.shuffle(1024).batch(64),
                    callbacks = [EarlyStopping(monitor = 'val_loss', patience=5),
                                 AgeAccCallback(val_new_batch),])
  
  # check whether metrics improved and save model
  new_mean_acc = (hist.history['val_gender_output_accuracy'][0] + hist.history['overall_age_accuracy'][0])/2
  gen_acc = hist.history['gender_output_accuracy'][0]
  if new_mean_acc > old_mean_acc:
    print('Validation mean accuracy for gender and age improved to ' + str(new_mean_acc) + ', saving model...')
    model2.save(file_path)
    old_mean_acc = new_mean_acc

Epoch 1/50
118/118 [==============================] - 76s 559ms/step - loss: 7.8224 - gender_output_loss: 0.6730 - age_group1_loss: 0.2029 - age_group2_loss: 0.4208 - age_group3_loss: 0.5699 - age_group4_loss: 0.6389 - age_group5_loss: 0.6475 - age_group6_loss: 0.5952 - age_group7_loss: 0.3725 - age_group8_loss: 0.3195 - gender_output_accuracy: 0.6042 - age_group1_accuracy: 0.9596 - age_group2_accuracy: 0.8501 - age_group3_accuracy: 0.7201 - age_group4_accuracy: 0.6498 - age_group5_accuracy: 0.6515 - age_group6_accuracy: 0.7125 - age_group7_accuracy: 0.8794 - age_group8_accuracy: 0.8992 - val_loss: 5.4798 - val_gender_output_loss: 0.6062 - val_age_group1_loss: 0.0645 - val_age_group2_loss: 0.1790 - val_age_group3_loss: 0.2875 - val_age_group4_loss: 0.3639 - val_age_group5_loss: 0.4189 - val_age_group6_loss: 0.4959 - val_age_group7_loss: 0.2706 - val_age_group8_loss: 0.1930 - val_gender_output_accuracy: 0.6768 - val_age_group1_accuracy: 1.0000 - val_age_group2_accuracy: 0.9771 - val_age

### Graphs for loss and accuracy against epochs

### Save and load trained model

In [6]:
model = tf.keras.models.load_model("/content/drive/MyDrive/data/saved_model/celeba_modified_age_gen_plus.h5")
model_full = tf.keras.models.load_model("/content/drive/MyDrive/data/saved_model/full_celeba_modified_age_gen_plus.h5")

In [7]:
model_full.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_4 (Functional)           (None, 7, 7, 512)    21302473    ['input_8[0][0]']                
                                                                                                  
 global_average_pooling2d_6 (Gl  (None, 512)         0           ['model_4[0][0]']                
 obalAveragePooling2D)                                                                            
                                                                                            

### Evaluate trained model on test data

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def gender_metrics(preds, ds):
  gender_true = []
  for i in ds.as_numpy_iterator():
    # take label dict
    label_dict = i[1] 
    # append all values with key 'gender_output' to a list
    gender = label_dict['gender_output'][0] 
    # append all gender to a list
    gender_true.append(gender) 
  gender_preds = np.where(preds[0].ravel()>0.5, 1, 0)
  return {'accuracy': accuracy_score(gender_true, gender_preds), 
          'f1': f1_score(gender_true, gender_preds), 
          'precision': precision_score(gender_true, gender_preds), 
          'recall': recall_score(gender_true, gender_preds)}

In [9]:
preds = model.predict(test_new_batch)

2244/2244 [==============================] - 33s 10ms/step


In [10]:
gender_metrics(preds, test_new_batch)

{'accuracy': 0.7330659536541889,
 'f1': 0.748846960167715,
 'precision': 0.7516835016835017,
 'recall': 0.746031746031746}

In [11]:
age_acc(model, test_new_batch)

0.3663101604278075

#### Test on model with all layers trainable

In [12]:
preds2 = model_full.predict(test_new_batch)

2244/2244 [==============================] - 22s 9ms/step


In [13]:
gender_metrics(preds2, test_new_batch)

{'accuracy': 0.7843137254901961,
 'f1': 0.7931623931623931,
 'precision': 0.8118985126859143,
 'recall': 0.7752715121136173}

In [14]:
age_acc(model_full, test_new_batch)

0.4585561497326203

In [15]:
preds3 = model_full.predict(val_new_batch)

2361/2361 [==============================] - 26s 11ms/step


In [16]:
gender_metrics(preds3, val_new_batch)

{'accuracy': 0.8513341804320204,
 'f1': 0.8626223091976516,
 'precision': 0.8496530454895914,
 'recall': 0.875993640699523}

In [17]:
age_acc(model_full, val_new_batch)

0.4849639983058026